# Data processing and empirical modelling

## Processing data in Julia

Different approaches are possible. We'll be using elements of [Tidier.jl](https://tidierorg.github.io/Tidier.jl/dev/), which mirrors a subset of the popular [Tidyverse](https://www.tidyverse.org/) collection of R packages.

In [1]:
using Tidier

### Reading in datasets

In [4]:
sales = read_csv("datasets/sales.csv")

Row,Customer_ID,Product_ID,Product_Name,Category,Unit_Price,Units_Sold,Revenue
,Int64,Int64,String15,String15,Int64,Int64,Int64
1,101,1,Laptop,Electronics,800,100,80000
2,102,2,T-Shirt,Clothing,20,500,10000
3,103,3,Smartphone,Electronics,600,200,120000
4,101,4,Sneakers,Clothing,50,300,15000
5,104,5,Milk,Groceries,2,1000,2000


In [5]:
customers =  read_csv("datasets/customers.csv")

Row,Customer_ID,Age,Gender,Income_Range,Region
,Int64,Int64,String7,String15,String7
1,101,35,Male,$50000-$75000,North
2,102,28,Female,$40000-$50000,South
3,103,45,Female,$75000-$100000,East
4,104,50,Male,$100000+,West
5,105,40,Male,$50000-$75000,North


### Selecting variables

In [6]:
sls = @chain sales begin
    @select(Product_ID,Product_Name,Unit_Price,Revenue,Units_Sold)
end

Row,Product_ID,Product_Name,Unit_Price,Revenue,Units_Sold
,Int64,String15,Int64,Int64,Int64
1,1,Laptop,800,80000,100
2,2,T-Shirt,20,10000,500
3,3,Smartphone,600,120000,200
4,4,Sneakers,50,15000,300
5,5,Milk,2,2000,1000


In [7]:
# Also possible
@select(sales, Product_ID,Product_Name,Unit_Price,Revenue,Units_Sold)

Row,Product_ID,Product_Name,Unit_Price,Revenue,Units_Sold
,Int64,String15,Int64,Int64,Int64
1,1,Laptop,800,80000,100
2,2,T-Shirt,20,10000,500
3,3,Smartphone,600,120000,200
4,4,Sneakers,50,15000,300
5,5,Milk,2,2000,1000


In [8]:
@chain sales begin
    @select(-Customer_ID) # exclude a column
end

Row,Product_ID,Product_Name,Category,Unit_Price,Units_Sold,Revenue
,Int64,String15,String15,Int64,Int64,Int64
1,1,Laptop,Electronics,800,100,80000
2,2,T-Shirt,Clothing,20,500,10000
3,3,Smartphone,Electronics,600,200,120000
4,4,Sneakers,Clothing,50,300,15000
5,5,Milk,Groceries,2,1000,2000


In [9]:
@chain sales begin
    @select(Category:Revenue) # select a subset of columns using a slice
end

Row,Category,Unit_Price,Units_Sold,Revenue
,String15,Int64,Int64,Int64
1,Electronics,800,100,80000
2,Clothing,20,500,10000
3,Electronics,600,200,120000
4,Clothing,50,300,15000
5,Groceries,2,1000,2000


### Filtering and slicing data. Renaming variables

In [10]:
@chain sales begin
    @filter(Category == "Electronics", Revenue >= 100000)
    @rename(Price = Unit_Price, Units = Units_Sold)
end

Row,Customer_ID,Product_ID,Product_Name,Category,Price,Units,Revenue
,Int64,Int64,String15,String15,Int64,Int64,Int64
1,103,3,Smartphone,Electronics,600,200,120000


In [11]:
@chain sales begin
    @slice(1:2,5)
end

Row,Customer_ID,Product_ID,Product_Name,Category,Unit_Price,Units_Sold,Revenue
,Int64,Int64,String15,String15,Int64,Int64,Int64
1,101,1,Laptop,Electronics,800,100,80000
2,102,2,T-Shirt,Clothing,20,500,10000
3,104,5,Milk,Groceries,2,1000,2000


### Mutating variables

In [12]:
sls = @chain sls begin
        @mutate(P = Revenue/Units_Sold)
        @mutate(check = P - Unit_Price)
end

Row,Product_ID,Product_Name,Unit_Price,Revenue,Units_Sold,P,check
,Int64,String15,Int64,Int64,Int64,Float64,Float64
1,1,Laptop,800,80000,100,800.0,0.0
2,2,T-Shirt,20,10000,500,20.0,0.0
3,3,Smartphone,600,120000,200,600.0,0.0
4,4,Sneakers,50,15000,300,50.0,0.0
5,5,Milk,2,2000,1000,2.0,0.0


In [13]:
@chain sls begin
        @transmute(problem = check!=0)
end

Row,problem
,Bool
1,false
2,false
3,false
4,false
5,false


### Joins

In [14]:
# left_join: first dataframe is the "leading" one
@chain sales begin
    @left_join(customers) # the common key is automatically discovered
end

Row,Customer_ID,Product_ID,Product_Name,Category,Unit_Price,Units_Sold,Revenue,Age,Gender,Income_Range,Region
,Int64,Int64,String15,String15,Int64,Int64,Int64,Int64?,String7,String15,String7
1,101,1,Laptop,Electronics,800,100,80000,35,Male,$50000-$75000,North
2,101,4,Sneakers,Clothing,50,300,15000,35,Male,$50000-$75000,North
3,102,2,T-Shirt,Clothing,20,500,10000,28,Female,$40000-$50000,South
4,103,3,Smartphone,Electronics,600,200,120000,45,Female,$75000-$100000,East
5,104,5,Milk,Groceries,2,1000,2000,50,Male,$100000+,West


In [15]:
# right_join: second dataframe is the "leading" one
@chain sales begin
    @right_join(customers) 
end

Row,Customer_ID,Product_ID,Product_Name,Category,Unit_Price,Units_Sold,Revenue,Age,Gender,Income_Range,Region
,Int64,Int64?,String15,String15,Int64?,Int64?,Int64?,Int64,String7,String15,String7
1,101,1,Laptop,Electronics,800,100,80000,35,Male,$50000-$75000,North
2,101,4,Sneakers,Clothing,50,300,15000,35,Male,$50000-$75000,North
3,102,2,T-Shirt,Clothing,20,500,10000,28,Female,$40000-$50000,South
4,103,3,Smartphone,Electronics,600,200,120000,45,Female,$75000-$100000,East
5,104,5,Milk,Groceries,2,1000,2000,50,Male,$100000+,West
6,105,missing,missing,missing,missing,missing,missing,40,Male,$50000-$75000,North


In [16]:
# inner_join: on common keys, here happens to be the same as the left_join
@chain sales begin
    @inner_join(customers) 
end

Row,Customer_ID,Product_ID,Product_Name,Category,Unit_Price,Units_Sold,Revenue,Age,Gender,Income_Range,Region
,Int64,Int64,String15,String15,Int64,Int64,Int64,Int64,String7,String15,String7
1,101,1,Laptop,Electronics,800,100,80000,35,Male,$50000-$75000,North
2,101,4,Sneakers,Clothing,50,300,15000,35,Male,$50000-$75000,North
3,102,2,T-Shirt,Clothing,20,500,10000,28,Female,$40000-$50000,South
4,103,3,Smartphone,Electronics,600,200,120000,45,Female,$75000-$100000,East
5,104,5,Milk,Groceries,2,1000,2000,50,Male,$100000+,West


### Groupby and summarize operations

In [17]:
@chain sales begin
    @group_by(Category)
    @mutate(avgP = mean(Unit_Price))
    @ungroup()
end

Row,Customer_ID,Product_ID,Product_Name,Category,Unit_Price,Units_Sold,Revenue,avgP
,Int64,Int64,String15,String15,Int64,Int64,Int64,Float64
1,101,1,Laptop,Electronics,800,100,80000,700.0
2,103,3,Smartphone,Electronics,600,200,120000,700.0
3,102,2,T-Shirt,Clothing,20,500,10000,35.0
4,101,4,Sneakers,Clothing,50,300,15000,35.0
5,104,5,Milk,Groceries,2,1000,2000,2.0


In [18]:
@chain sales begin
    @summarize(maxrev = maximum(Revenue), minP = minimum(Unit_Price))
end

Row,maxrev,minP
,Int64,Int64
1,120000,2


In [19]:
@chain sales begin
    @group_by(Customer_ID)
    @summarize(maxrev = maximum(Revenue), minP = minimum(Unit_Price))
end

Row,Customer_ID,maxrev,minP
,Int64,Int64,Int64
1,101,80000,50
2,102,10000,20
3,103,120000,600
4,104,2000,2


### Pivot_wider and pivot_longer

In [20]:
@chain sales begin
    @select(Product_Name, Units_Sold)
    @pivot_wider(names_from = Product_Name, values_from = Units_Sold)
end

Row,Laptop,T-Shirt,Smartphone,Sneakers,Milk
,Int64?,Int64?,Int64?,Int64?,Int64?
1,100,500,200,300,1000


In [21]:
@chain sales begin
    @select(Product_ID, Category, Units_Sold)
    @pivot_wider(names_from = Category, values_from = Units_Sold)
end

Row,Product_ID,Electronics,Clothing,Groceries
,Int64,Int64?,Int64?,Int64?
1,1,100,missing,missing
2,2,missing,500,missing
3,3,200,missing,missing
4,4,missing,300,missing
5,5,missing,missing,1000


In [22]:
@chain sales begin
    @select(Customer_ID, Unit_Price, Units_Sold)
    @pivot_longer(Unit_Price:Units_Sold, names_to = "type", values_to = "value")
end

Row,Customer_ID,type,value
,Int64,String,Int64
1,101,Unit_Price,800
2,102,Unit_Price,20
3,103,Unit_Price,600
4,101,Unit_Price,50
5,104,Unit_Price,2
6,101,Units_Sold,100
7,102,Units_Sold,500
8,103,Units_Sold,200
9,101,Units_Sold,300


# Growth accounting

Consider a two-factor Cobb-Douglas production function of the form
$$ Y_t = A_t K_t^\alpha L_t^{1-\alpha} $$

Here $Y_t$ is output (real GDP) in period $t$, $L_t$ is employment, $K_t$ is the capital stock and $A_t$ is total factor productivity (technology). The parameter $\alpha$ will be assumed to be known. Typical estimates are derived from national accounts data and can range from 0.3 to 0.45.

Data for $Y_t$ and $L_t$ is directly available from statistical sources (e.g. Eurostat). 

While data on $K_t$ is not published directly, there are techniques to reconstruct the capital stock under appropriate assumptions.

Then, TFP can be computed as a residual, e.g.
$$ \ln Y_t = \ln A_t + \alpha \ln K_t + (1-\alpha) \ln L_t ~\Rightarrow~ a_t = y_t - \alpha k_t -(1-\alpha)l_t,$$
where lower-case letters denote logs. This is known as the **Solow residual**.

Because a production function approach provides only an approximation to the real world, what actually happens is that $A_t$, measured as a residual picks up a lot of other influences, so it can't directly be interpreted as a measure of technology. This makes it interesting and a subject to independent analyses.

## Process data on GDP and components

In [23]:
natl_acc = read_csv("datasets/nama_10_gdp_linear.csv.gz")

Row,DATAFLOW,LAST UPDATE,freq,unit,na_item,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
,String31,String31,String1,String31,String15,String15,Int64,Float64?,String1
1,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,1995,177617.0,missing
2,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,1996,1.80999e5,missing
3,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,1997,1.84799e5,missing
4,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,1998,1.92025e5,missing
5,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,1999,1.98247e5,missing
6,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,2000,2.05898e5,missing
7,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,2001,2.08772e5,missing
8,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,2002,2.12101e5,missing
9,ESTAT:NAMA_10_GDP(1.0),20/04/24 11:00:00,A,CLV05_MEUR,B1G,AT,2003,214463.0,missing


In [24]:
nat_acc = @chain natl_acc begin
    @select(unit, na_item, geo, TIME_PERIOD,OBS_VALUE)
    @rename(year=TIME_PERIOD, value=OBS_VALUE)
end

Row,unit,na_item,geo,year,value
,String31,String15,String15,Int64,Float64?
1,CLV05_MEUR,B1G,AT,1995,177617.0
2,CLV05_MEUR,B1G,AT,1996,1.80999e5
3,CLV05_MEUR,B1G,AT,1997,1.84799e5
4,CLV05_MEUR,B1G,AT,1998,1.92025e5
5,CLV05_MEUR,B1G,AT,1999,1.98247e5
6,CLV05_MEUR,B1G,AT,2000,2.05898e5
7,CLV05_MEUR,B1G,AT,2001,2.08772e5
8,CLV05_MEUR,B1G,AT,2002,2.12101e5
9,CLV05_MEUR,B1G,AT,2003,214463.0


In [25]:
bg_nat_acc = @chain nat_acc begin
    @filter(geo == "BG", unit == "CLV10_MNAC", na_item in ("B1GQ", "P51G"))
    @select(na_item,year,value)
    @drop_missing()
    @pivot_wider(names_from = na_item, values_from = value)
    @rename(gdp = B1GQ, inv = P51G)
end

Row,year,gdp,inv
,Int64,Float64?,Float64?
1,1995,53253.3,4874.4
2,1996,56025.4,1575.2
3,1997,48117.2,3945.4
4,1998,49940.4,5232.7
5,1999,45747.3,6404.1
6,2000,47845.9,7459.4
7,2001,49675.4,8938.7
8,2002,52592.3,9665.9
9,2003,55346.6,10908.3


## Process data on employment

In [26]:
labour = read_csv("datasets/nama_10_pe_linear.csv.gz")

Row,DATAFLOW,LAST UPDATE,freq,unit,na_item,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
,String31,String31,String1,String15,String7,String15,Int64,Float64?,String1
1,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,1996,0.4,missing
2,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,1997,0.7,missing
3,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,1998,1.0,missing
4,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,1999,1.5,missing
5,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,2000,0.9,missing
6,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,2001,0.7,missing
7,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,2002,-0.1,missing
8,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,2003,0.7,missing
9,ESTAT:NAMA_10_PE(1.0),20/04/24 11:00:00,A,PCH_PRE_PER,EMP_DC,AT,2004,0.6,missing


In [27]:
lab = @chain labour begin
    @select(unit, na_item, geo, TIME_PERIOD,OBS_VALUE)
    @rename(year=TIME_PERIOD, value=OBS_VALUE)
end

Row,unit,na_item,geo,year,value
,String15,String7,String15,Int64,Float64?
1,PCH_PRE_PER,EMP_DC,AT,1996,0.4
2,PCH_PRE_PER,EMP_DC,AT,1997,0.7
3,PCH_PRE_PER,EMP_DC,AT,1998,1.0
4,PCH_PRE_PER,EMP_DC,AT,1999,1.5
5,PCH_PRE_PER,EMP_DC,AT,2000,0.9
6,PCH_PRE_PER,EMP_DC,AT,2001,0.7
7,PCH_PRE_PER,EMP_DC,AT,2002,-0.1
8,PCH_PRE_PER,EMP_DC,AT,2003,0.7
9,PCH_PRE_PER,EMP_DC,AT,2004,0.6


In [28]:
bg_lab = @chain lab begin
    @filter(geo == "BG", unit ==  "THS_PER", na_item == "EMP_DC")
    @select(year,value)
    @drop_missing()
    @rename(emp = value)
end

Row,year,emp
,Int64,Float64
1,1995,3518.57
2,1996,3614.98
3,1997,3503.91
4,1998,3467.92
5,1999,3318.15
6,2000,3239.2
7,2001,3214.74
8,2002,3222.12
9,2003,3317.39


## Combine output, investment and employment

In [29]:
gracc = @chain bg_nat_acc begin
    @inner_join(bg_lab)
end

Row,year,gdp,inv,emp
,Int64,Float64?,Float64?,Float64
1,1995,53253.3,4874.4,3518.57
2,1996,56025.4,1575.2,3614.98
3,1997,48117.2,3945.4,3503.91
4,1998,49940.4,5232.7,3467.92
5,1999,45747.3,6404.1,3318.15
6,2000,47845.9,7459.4,3239.2
7,2001,49675.4,8938.7,3214.74
8,2002,52592.3,9665.9,3222.12
9,2003,55346.6,10908.3,3317.39


### Perpetual inventory method

